In [1]:
# import libraries
from __future__ import print_function

import pandas as pd
import numpy as np

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

from collections import Counter, defaultdict

import time

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
# if 'autoreload' not in ipython.extension_manager.loaded:
#     get_ipython().run_line_magic('load_ext', 'autoreload')

# get_ipython().run_line_magic('autoreload', '2')

# Visualizations
import seaborn as sns

In [2]:
def compute_tf(word_dict, l): # l is the message from logs
            tf = {}
            sum_nk = len(l)
            for word, count in word_dict.items():
                try:
                    tf[word] = count/sum_nk
                except ZeroDivisionError:
                    tf[word] = 0
            return tf
        
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1

    for word, v in idf.items():
        idf[word] = np.log(n / float(v))
    return idf

def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf
      

def Vectorisation(file_number):
    data_set = []
    for v in file_number:
        file_name = f"../File/FrontendFileGroup/storm-frontend-202003{v}-mask-group.txt"

        logs = pd.read_csv(file_name, index_col=0, nrows=1e4)
        tokens_per_message = [x.lower().split() for x in logs.message]
        word_set = set()
        
        for mess in tokens_per_message:
            word_set = word_set.union(set(mess))

        print("We have {} logs messages, for a total of {} unique tokens adopted.".format(
            len(tokens_per_message), len(word_set)))

        word_dict = [dict.fromkeys(word_set, 0) for i in range(len(tokens_per_message))]

        # Compute raw frequencies of each token per each message
        for i in range(len(logs.message)):
            for word in tokens_per_message[i]:
                word_dict[i][word] += 1

        c = 0
        for i, dic in enumerate(tokens_per_message):
            if not len(dic):
                print(i, errors.loc[i])
                c += 1

        print("Warning: there are {} blanck messages which will be excluded from the analysis.".format(c))

        tf = [compute_tf(word_dict[i], tokens_per_message[i])
              for i in range(len(tokens_per_message))] #if sum(word_dict[i].values())]

        idf = compute_idf(word_dict)

        tf_idf =  [compute_tf_idf(tf[i], idf) for i in range(len(tf))]

        # Extract TF-IDF information
        print("Extracting features from the training dataset using a sparse vectorizer")
        t0 = time()
        vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.02, stop_words='english',
                                     use_idf=True)
        # vectorizer = TfidfVectorizer(stop_words='english',
        #                              use_idf=True)
        X = vectorizer.fit_transform(logs.message)

        print("done in %fs" % (time() - t0))
        print("n_samples: %d, n_features: %d" % X.shape)
        print()

        # Apply LSA for dimensionality reduction to get a lower-dimensional embedding space
        print("Performing dimensionality reduction using LSA")
        t0 = time()

        # Vectorizer results are normalized, which makes KMeans behave as
        # spherical k-means for better results. Since LSA/SVD results are
        # not normalized, we have to redo the normalization.
        svd = TruncatedSVD(25)
        normalizer = Normalizer(copy=False)
        lsa = make_pipeline(svd, normalizer)
        X = lsa.fit_transform(X)

        print("done in %fs" % (time() - t0))

        explained_variance = svd.explained_variance_ratio_.sum()
        print("Explained variance of the SVD step: {}%".format(
            int(explained_variance * 100)))
        
        data_set.append(X)
        print()
    
    return data_set

In [3]:
file_number = ['07', '08']

In [ ]:
data_set = Vectorisation(file_number)

We have 10000 logs messages, for a total of 10124 unique tokens adopted.
Extracting features from the training dataset using a sparse vectorizer


In [ ]:
data_set[0]

In [ ]:
meanErr = []
for v in file_number:
    file_name = f"../File/FrontendFileGroup/storm-frontend-202003{v}-mask-group.txt"
    logs = pd.read_csv(file_name, index_col=0, nrows=1e4)
    
    X = data_set[file_number.index(v)]
    # run K-Means algorithm: 6 clusters
    km = KMeans(n_clusters=28, 
                init='k-means++', 
                max_iter=500, 
                n_init=100,
#                 verbose=1
               )

    print("Clustering sparse data with %s" % km)
    t0 = time()
    km.fit(X)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("We have {} centroids represented as {}-dimensional points.".format(km.cluster_centers_.shape[0],
                                                                              km.cluster_centers_.shape[1]))    
    # print the numerosity of each cluster
#     print(Counter(km.labels_))

    logs["kmean_labels"] = km.labels_

    label = np.unique(km.labels_)
    count = [Counter(km.labels_)[i] for i in label]

    # plot:
    fig, ax = plt.subplots(figsize = (8, int(len(label)/3)))

    for l, c in zip(label, count):
        ax.barh(l, c, linewidth=0.5, edgecolor="white", label=logs.message[logs.kmean_labels==l][0])
        ax.text(10, l-0.1, logs.message[logs.kmean_labels==l][0][:80]
#                 +'\n'+logs.message[logs.kmean_labels==l][0][30:60]
               )

#     with open(f'./BestCentroid/frontend-202003{v}-{n_clusters}centroids.txt', 'w') as fwrite:
#         for l in label:
#             fwrite.write(f'Label {l}:\n')
#             for i in range(5):
#                 fwrite.write(logs.message[logs.kmean_labels==l][i])
#                 fwrite.write('\n')
#             fwrite.write('\n')
            
    ax.set(yticks=label)

    plt.title(f'frontend-202003{v}')
    plt.savefig(f'./BestCentroid/frontend-202003{v}', bbox_inches ="tight", facecolor='white')
    
    
    import re

    error = [0] * len(label)

    for l in label:
        for msg in logs.message[logs.kmean_labels==l]:
            resultE = re.search('error', msg.lower())
            resultF = re.search('failure', msg.lower())
            result = re.search('SRM_FAILURE', msg)
            if resultE!=None :
                error[l] += 1
            if resultF!=None:
                error[l] += 1
            if result!=None:
                error[l] += 1

            
    # plot:
    fig, ax = plt.subplots(figsize = (8, int(len(label)/3)))
    
    error_mean = np.mean(error)
    for l, c in zip(label, count):
        if error[l] > error_mean:
            color='red'
        else:
            color='green'
        ax.barh(l, c, linewidth=0.5, color=color, edgecolor="white", label=logs.message[logs.kmean_labels==l][0])
        ax.text(10, l-0.2, logs.message[logs.kmean_labels==l][0][:80]
#                 +'\n'+logs.message[logs.kmean_labels==l][0][30:60]
               )

    # ax.set(xticks=np.arange(0, max(count), 10), yticks=label)
    ax.set(yticks=label)
    ax.set
    # ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
    #        ylim=(0, 56), yticks=np.linspace(0, 56, 9))



    # ax.legend()
    plt.title(f'frontend-202003{v} '+str(error))
#     plt.savefig(f'frontend-202003{v}-{n_clusters}centroids', bbox_inches ="tight")
    plt.show()
    meanErr.append(np.mean(error))


In [ ]:
meanErr

In [ ]:
np.mean(meanErr)